### Pool function in the multiple imputation

2019-05-12

Just dobule check

The MI inference is usually based on a $t$-distribution with DF given by 
$$v = (Q - 1) \big(1 + \frac{Q}{Q+1} \frac{W}{B}\big)^2$$
where, 

* B: between-imputation variance 
* w: average within-imputation variance
* Q: number of imputation

Howeverm this DF is derived under the assumpition that the complete data's DF ($v_{\text{com}}$) is infinite. 
<br>
The value $v_{\text{com}}$ is usually calculated based on the number of clusters in the study, rather than the number of individuals. In CRT, with equal number of clusters in each intervention group, $v_{\text{com}}$ can be calculated as 
$$v_{\text{com}} = 2k - 2$$

If $k$ is small, then it is not appropriate. We need to calculate the adjusted DF. It can be writen as: 

$$v_{\text{adj}} = (v^{-1} + \hat{v}^{-1}_{\text{obs}})^{-1}$$
where
$$\hat{v}^{-1}_{\text{obs}} = \big(\frac{v_{\text{com}} + 1}{v_{\text{com}} + 3}\big) v_{\text{com}} \big( 1 + \frac{Q + 1}{Q}\frac{B}{W}\big)^{-1}$$

#### The R function:

In [1]:
mypool <- function(mean0,sd0,num=5,J=50){
 
  ########################################
  # input: 
  # mean0: a vector of the values of estimated beta parameter, with length equals to the imputation time.
  # sd0: a vector of the standard deviations of the estimated beta parameter, with length equals to the imputation time.
  # num: impuation time
  # J: the number of clusters in each intervention arm 
  ########################################
  
  # count the times of NA in the input vector.
  na_times <- sum(is.na(mean0)) 
  # the number of imputations without NA. 
  num_actual <- num-na_times 
  
  # the MI estimate of the beta parameter
  m <- mean(mean0,na.rm=TRUE) 
  
  # estimate of average wihtin-imputation variance 
  # i.e. based on the SE^2 of the beta parameter from each fitted model
  W <- mean(sd0^2,na.rm=TRUE) 
  
  # estimate of between-imputation variance 
  # i.e. empirical SD of the point estimates of beta parameter
  B <- var(mean0,na.rm=TRUE) 
  
  # estimate of total variance 
  # i.e. will need to take the sqrt of this to use for inference in making confidence intervals etc.
  v_hat <- W+(1+1/num_actual)*B 
  v_hat_sqrt<-sqrt(v_hat)
  
  # Testing based on standard results from MI literature
  # i.e. df of t distribution for testing based on standard results from MI literature
  df_denom <- (1+1/num_actual)*B
  df_part <- 1+W/df_denom
  df_t <- (num_actual-1)*df_part^2 # adjusted df of t distribution
 
  # Testing based on results from MMI literature, Barnard and Rubin (1999), 
  # df of t distribution for testing based on results in re adjustment for MMI feature
  
  #df for full data
  df_com <- 2*J - 2 
  
  # calculate the adjusted df based on the literature.
  parenthesis <- 1+df_denom*(1/W) 
  df_obs <- df_com*((df_com+1)/(df_com+3))*(1/parenthesis) 
  df_adj_t <- 1/(1/df_t + 1/df_obs) 
  
  # Print the results
  print("Standard t df and Barnard/Rubin adjusted t df");
  print(c(df_t, df_adj_t))
  print("97.5% quantiles from standard t df and Barnard/Rubin adjusted t df");
  print(c(qt(0.975,df_t), qt(0.975,df_adj_t)))
  
  return(list(mean=m,std=v_hat_sqrt,
              df_t=df_t,
              df_adj_t=df_adj_t))
}


In [2]:
mean0 = rnorm(15); sd0 = rnorm(15); num = 15; J = 10

The $Q$ is: 

In [3]:
# the number of imputations without NA. 
na_times <- sum(is.na(mean0)) 
num_actual <- num-na_times 
num_actual

[1] 15

Estimate of average wihtin-imputation variance 

In [4]:
W <- mean(sd0^2,na.rm=TRUE) 
W

[1] 1.192358

Estimate of between-imputation variance 

In [5]:
B <- var(mean0,na.rm=TRUE) 
B

[1] 1.053913

Calculate $v$
$$v = (Q - 1) \big(1 + \frac{Q}{Q+1} \frac{W}{B}\big)^2$$

* $v = \text{df_t} = (\text{num_actual} - 1) (\text{df_part})^2$
* $\text{df_part} = \big(1 + \frac{Q}{Q+1} \frac{W}{B}\big) = (1+W/\text{df_denom})$
* $1/\text{df_denom} = \frac{Q}{Q+1} \frac{1}{B}$
* $\text{df_denom} = \frac{Q+1}{Q} B = (1 + \frac{1}{Q})B$

Therefore,

In [6]:
df_denom <- (1+1/num_actual)*B
df_part <- 1+W/df_denom
df_t <- (num_actual-1)*df_part^2
df_t

[1] 59.44805

Calculate
$v_{\text{com}} = 2k - 2$

In [7]:
df_com <- 2*J - 2 
df_com

[1] 18

Calculate 
$$\hat{v}^{-1}_{\text{obs}} = \big(\frac{v_{\text{com}} + 1}{v_{\text{com}} + 3}\big) v_{\text{com}} \big( 1 + \frac{Q + 1}{Q}\frac{B}{W}\big)^{-1}$$

* $\text{df_denom} = (1 + \frac{1}{Q})B$
* $\text{parenthesis}$ = $\big( 1 + \frac{Q + 1}{Q}\frac{B}{W}\big) = 1 + \text{df_denom}/W$


In [8]:
parenthesis <- 1+df_denom*(1/W)
parenthesis

[1] 1.942816

In [9]:
df_obs <- df_com*((df_com+1)/(df_com+3))*(1/parenthesis) 
df_obs

[1] 8.382532

Calculate the 
$$v_{\text{adj}} = (v^{-1} + \hat{v}^{-1}_{\text{obs}})^{-1}$$

In [10]:
df_adj_t <- 1/(1/df_t + 1/df_obs)
df_adj_t

[1] 7.346615